# Dataset Statistics

In [49]:
import os
import pandas
import numpy
from pandas.plotting import table
from matplotlib import pyplot
from Bio import SeqIO
from biotite.sequence.align import get_sequence_identity, get_pairwise_sequence_identity, align_optimal, SubstitutionMatrix
from biotite.sequence.io import fasta
from biotite.sequence import NucleotideSequence
from itertools import combinations
pandas.set_option('display.max_columns', None)

In [2]:
data_dir = "../../data"
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin", "indelible"]

In [5]:
def lcs(first, second):
    m = len(first.seq)
    n = len(second.seq)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_str = ""
    for i in range(m):
        for j in range(n):
            if first.seq[i] == second.seq[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_str = str(first.seq[i-c+1:i+1])
                    longest = c
    return lcs_str

In [6]:
def repeats(s, k=3):
    max_repeats = 0
    for i in range(0, len(s.seq), k):
        max_repeats = max(max_repeats, s.seq.count(s.seq[i:i+k]))
    return max_repeats

## Nucleotide

In [9]:
def compute_stats(dataset):
    fasta_file = f"{data_dir}/{dataset}.fasta.N.sanitized"
    seqs = list(SeqIO.parse(fasta_file, "fasta"))
    lcs_seqs = [len(lcs(s1, s2)) for (s1, s2) in combinations(seqs, 2)]
    repeats_len = [repeats(s) for s in seqs]
    seqs_len = [len(s.seq) for s in seqs]
    seqs_mean = numpy.mean(seqs_len)
    seqs_median = numpy.median(seqs_len)
    seqs_std = numpy.std(seqs_len)
    seqs_min = numpy.min(seqs_len)
    seqs_max = numpy.max(seqs_len)
    align = fasta.get_alignment(
        fasta.FastaFile.read(f"{data_dir}/trees/N/full/{dataset}/Control with Clustal Omega.fasta"))
    seqs_identity = get_sequence_identity(align)
    gaps = [seq.count("-") for seq in fasta.FastaFile.read(f"{data_dir}/trees/N/full/{dataset}/Control with Clustal Omega.fasta").values()]
    result_df = pandas.DataFrame(
        {"median": int(seqs_median),
         "mean": seqs_mean,
         "std": round(seqs_std, 2),
         "min": seqs_min,
         "max": seqs_max,
         "identity": seqs_identity,
         "median_rep": numpy.median(repeats_len),
         "mean_rep": numpy.mean(repeats_len),
         "std_rep": numpy.std(repeats_len),
         "min_rep": numpy.min(repeats_len),
         "max_rep": numpy.max(repeats_len),
         "median_lcs": numpy.median(lcs_seqs),
         "mean_lcs": numpy.mean(lcs_seqs),
         "std_lcs": numpy.std(lcs_seqs),
         "min_lcs": numpy.min(lcs_seqs),
         "max_lcs": numpy.max(lcs_seqs),
         "with_gaps": sum([1 if g else 0 for g in gaps]),
         "median_gaps": numpy.median(gaps),
         "mean_gaps": numpy.mean(gaps),
         "std_gaps": numpy.std(gaps),
         "min_gaps": numpy.min(gaps),
         "max_gaps": numpy.max(gaps),
         "sample size": len(seqs_len)}, index=[dataset])
    result_df.index.name = "dataset"
    return result_df

In [10]:
dfs = []
for dataset in datasets:
    try:
        dfs.append(compute_stats(dataset))
    except:
        print(dataset)
        raise
pandas.concat(dfs)

,median,mean,std,min,max,identity,median_rep,mean_rep,std_rep,min_rep,max_rep,median_lcs,mean_lcs,std_lcs,min_lcs,max_lcs,with_gaps,median_gaps,mean_gaps,std_gaps,min_gaps,max_gaps,sample size
dataset,,,,,,,,,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,441,441.0,0.00,441,441,0.750567,25.0,24.533333,1.257864,21,26,88.0,98.419048,88.533532,30,441,0,0.0,0.0,0.000000,0,0,15
orthologs_myoglobin,465,465.0,0.00,465,465,0.763441,22.0,21.533333,1.203698,19,23,65.0,74.247619,61.949101,22,465,0,0.0,0.0,0.000000,0,0,15
orthologs_neuroglobin,456,451.6,11.43,417,456,0.610422,24.0,23.133333,1.820867,19,25,77.0,107.828571,92.300901,32,456,2,0.0,4.4,11.429786,0,39,15
orthologs_cytoglobin,618,596.2,66.25,378,678,0.396465,25.0,24.333333,1.813529,20,27,113.0,125.209524,70.959102,20,404,15,66.0,87.8,66.249226,6,306,15
orthologs_androglobin,4929,4726.4,694.56,2148,5004,0.605203,190.0,179.000000,31.385772,64,194,130.0,175.838095,194.554393,43,1477,15,125.0,327.6,694.563057,50,2906,15
indelible,3000,3000.0,0.00,3000,3000,0.000000,64.0,63.375000,3.336821,57,71,11.0,11.365385,1.056643,10,17,40,3302.0,3302.0,0.000000,3302,3302,40


### Just Intelible

In [3]:
fasta_file = f"{data_dir}/indelible.fasta.N.sanitized"
seqs = list(SeqIO.parse(fasta_file, "fasta"))
matrix = SubstitutionMatrix.std_nucleotide_matrix()
pair_identity = dict()
for (s1, s2) in combinations(seqs, 2):
    s1_desc, s2_desc = s1.description, s2.description
    aln = align_optimal(NucleotideSequence(s1.seq), NucleotideSequence(s2.seq), matrix)
    identity = get_sequence_identity(aln[0])
    pair_identity[s1_desc, s2_desc] = pair_identity[s2_desc, s1_desc] = identity
    print(s1_desc, s2_desc, identity)

A0 B0 0.6109832292009207
A0 C0 0.5210862619808306
A0 D0 0.5092063492063492
A0 A1 0.506716651046548
A0 B1 0.5023474178403756
A0 C1 0.5048452641450454
A0 D1 0.5031152647975078
A0 A2 0.5059375
A0 B2 0.4949653870358716
A0 C2 0.5007819831091649
A0 D2 0.5031308703819661
A0 A3 0.49482596425211667
A0 B3 0.4951425885302413
A0 C3 0.5035725380552967
A0 D3 0.495435945860875
A0 A4 0.4894221660877802
A0 B4 0.49546165884194054
A0 C4 0.49292675259352403
A0 D4 0.4927672955974843
A0 A5 0.5056390977443609
A0 B5 0.507327720611163
A0 C5 0.5090455396132252
A0 D5 0.5003130870381967
A0 A6 0.5067082683307332
A0 B6 0.5065625
A0 C6 0.5048392132375897
A0 D6 0.5001569858712716
A0 A7 0.5031387319522913
A0 B7 0.5020357031005324
A0 C7 0.5010975227343994
A0 D7 0.49372253609541744
A0 A8 0.4949685534591195
A0 B8 0.5048301651604862
A0 C8 0.5098100280286515
A0 D8 0.5042016806722689
A0 A9 0.49858712715855574
A0 B9 0.5056004978220286
A0 C9 0.49371464487743555
A0 D9 0.49764816556914393
B0 C0 0.5154213036565978
B0 D0 0.510204

B2 C7 0.5087390761548065
B2 D7 0.49968652037617556
B2 A8 0.5076682316118936
B2 B8 0.509386733416771
B2 C8 0.4963961140708242
B2 D8 0.49182903834066627
B2 A9 0.506574827802129
B2 B9 0.5014066895904971
B2 C9 0.5046992481203008
B2 D9 0.5009386733416771
C2 D2 0.5905073649754501
C2 A3 0.500156201187129
C2 B3 0.5039025913206369
C2 C3 0.4973278843131091
C2 D3 0.5037429819089208
C2 A4 0.5018714909544604
C2 B4 0.5003138731952291
C2 C4 0.49952874646559847
C2 D4 0.5082734935997503
C2 A5 0.5070334479524852
C2 B5 0.5073322932917317
C2 C5 0.5035903840149859
C2 D5 0.4989004084197298
C2 A6 0.5059337913803873
C2 B6 0.5035948733979368
C2 C6 0.5057939242092077
C2 D6 0.5075140889167189
C2 A7 0.5021916092673764
C2 B7 0.5001564945226917
C2 C7 0.49607535321821034
C2 D7 0.49843260188087773
C2 A8 0.5001568873548792
C2 B8 0.5059263880224579
C2 C8 0.4932496075353218
C2 D8 0.5029641185647425
C2 A9 0.5043942247332078
C2 B9 0.5025062656641605
C2 C9 0.49355953502984606
C2 D9 0.5010954616588419
D2 A3 0.5090625
D2 B3 

A6 C8 0.49480968858131485
A6 D8 0.493730407523511
A6 A9 0.49008498583569404
A6 B9 0.49402139710509757
A6 C9 0.5021943573667712
A6 D9 0.5023415547923822
B6 C6 0.51629392971246
B6 D6 0.5159134309357097
B6 A7 0.5009386733416771
B6 B7 0.5092101155167031
B6 C7 0.5075
B6 D7 0.4996869129618034
B6 A8 0.5107711520449578
B6 B8 0.5025015634771732
B6 C8 0.4932496075353218
B6 D8 0.4959170854271357
B6 A9 0.5007824726134585
B6 B9 0.4948194662480377
B6 C9 0.5122861586314152
B6 D9 0.5048513302034429
C6 D6 0.6003930560104815
C6 A7 0.49780564263322885
C6 B7 0.5035926273039675
C6 C7 0.5096453018046049
C6 D7 0.4957587181903864
C6 A8 0.5121495327102804
C6 B8 0.5039123630672926
C6 C8 0.5003128911138923
C6 D8 0.49953168904152356
C6 A9 0.49859066708424676
C6 B9 0.5063763608087092
C6 C9 0.49133858267716535
C6 D9 0.5067334794863765
D6 A7 0.5062460961898814
D6 B7 0.50109683484801
D6 C7 0.5085589791472145
D6 D7 0.4962264150943396
D6 A8 0.5107981220657277
D6 B8 0.5084586466165414
D6 C8 0.5101340816962894
D6 D8 0.50

In [84]:
group_combinations, extra_combinations = dict(), dict()
for i in range(10):
    intra = [f"A{i}", f"B{i}", f"C{i}", f"D{i}"]
    group_combinations[i] = []
    extra_combinations[i] = []
    for (s1, s2) in combinations(intra, 2):
        group_combinations[i].append(pair_identity[s1, s2])
    for (p1, p2) in pair_identity:
        if (p1 in intra) and (p2 in intra):
            pass
        elif (p1 in intra) or (p2 in intra):
            extra_combinations[i].append(pair_identity[p1, p2])
rows = list()
for i in range(10):
    rows.append((i, numpy.mean(group_combinations[i]), numpy.mean(extra_combinations[i]), numpy.median(group_combinations[i]), numpy.median(extra_combinations[i])))

In [92]:
df_groups = pandas.DataFrame(rows, columns=["Dados", "Média Intra", "Média Extra", "Mediana Intra", "Mediana Extra"]).set_index("Dados").style.set_precision(3)
df_groups

/tmp/ipykernel_477797/3782467938.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df_groups = pandas.DataFrame(rows, columns=["Dados", "Média Intra", "Média Extra", "Mediana Intra", "Mediana Extra"]).set_index("Dados").style.set_precision(3)


,Média Intra,Média Extra,Mediana Intra,Mediana Extra
Dados,,,,
0,0.544,0.502,0.518,0.502
1,0.547,0.503,0.520,0.502
2,0.542,0.502,0.515,0.502
3,0.542,0.500,0.515,0.500
4,0.545,0.500,0.516,0.500
5,0.548,0.502,0.522,0.502
6,0.544,0.503,0.516,0.503
7,0.541,0.501,0.514,0.501
8,0.548,0.502,0.522,0.502


In [94]:
print(df_groups.to_latex())

\begin{tabular}{lrrrr}
 & Média Intra & Média Extra & Mediana Intra & Mediana Extra \\
Dados &  &  &  &  \\
0 & 0.544 & 0.502 & 0.518 & 0.502 \\
1 & 0.547 & 0.503 & 0.520 & 0.502 \\
2 & 0.542 & 0.502 & 0.515 & 0.502 \\
3 & 0.542 & 0.500 & 0.515 & 0.500 \\
4 & 0.545 & 0.500 & 0.516 & 0.500 \\
5 & 0.548 & 0.502 & 0.522 & 0.502 \\
6 & 0.544 & 0.503 & 0.516 & 0.503 \\
7 & 0.541 & 0.501 & 0.514 & 0.501 \\
8 & 0.548 & 0.502 & 0.522 & 0.502 \\
9 & 0.544 & 0.502 & 0.522 & 0.502 \\
\end{tabular}



In [5]:
df = pandas.DataFrame.from_dict(pair_identity.items())
df[["seq1", "seq2"]] = df[0].tolist() 

In [6]:
df.columns = ["old", "score", "seq1", "seq2"]
df = df[["score", "seq1", "seq2"]]
df.head()

,score,seq1,seq2
0,0.610983,A0,B0
1,0.610983,B0,A0
2,0.521086,A0,C0
3,0.521086,C0,A0
4,0.509206,A0,D0


In [39]:
df_pivot = df.pivot(columns="seq1", index="seq2", values="score")
df_pivot_filled = df_pivot.fillna(1)

In [40]:
df_pivot_filled = df_pivot_filled.sort_index(axis=1, key=lambda x: x.str[-1]+x.str[0]).sort_index(axis=0, key=lambda x: x.str[-1]+x.str[0])

In [81]:
df_plot = df_pivot_filled.iloc[12:20,12:20].style.background_gradient(cmap='coolwarm', vmin=df_pivot.min().min(), vmax=df_pivot.max().max()+0.009).set_precision(3)#.set_properties(**{'font-size': '0pt'})
df_plot

/tmp/ipykernel_477797/3170969017.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df_plot = df_pivot_filled.iloc[12:20,12:20].style.background_gradient(cmap='coolwarm', vmin=df_pivot.min().min(), vmax=df_pivot.max().max()+0.009).set_precision(3)#.set_properties(**{'font-size': '0pt'})


seq1,A3,B3,C3,D3,A4,B4,C4,D4
seq2,,,,,,,,
A3,1.000,0.617,0.509,0.507,0.494,0.498,0.499,0.489
B3,0.617,1.000,0.520,0.506,0.494,0.499,0.493,0.493
C3,0.509,0.520,1.000,0.596,0.494,0.503,0.509,0.502
D3,0.507,0.506,0.596,1.000,0.488,0.503,0.499,0.494
A4,0.494,0.494,0.494,0.488,1.000,0.598,0.513,0.512
B4,0.498,0.499,0.503,0.503,0.598,1.000,0.514,0.517
C4,0.499,0.493,0.509,0.499,0.513,0.514,1.000,0.615
D4,0.489,0.493,0.502,0.494,0.512,0.517,0.615,1.000


In [82]:
print(df_plot.to_latex(convert_css=True))

\begin{tabular}{lrrrrrrrr}
seq1 & A3 & B3 & C3 & D3 & A4 & B4 & C4 & D4 \\
seq2 &  &  &  &  &  &  &  &  \\
A3 & {\cellcolor[HTML]{B40426}} \color[HTML]{F1F1F1} 1.000 & {\cellcolor[HTML]{D0473D}} \color[HTML]{F1F1F1} 0.617 & {\cellcolor[HTML]{6A8BEF}} \color[HTML]{F1F1F1} 0.509 & {\cellcolor[HTML]{6485EC}} \color[HTML]{F1F1F1} 0.507 & {\cellcolor[HTML]{465ECF}} \color[HTML]{F1F1F1} 0.494 & {\cellcolor[HTML]{506BDA}} \color[HTML]{F1F1F1} 0.498 & {\cellcolor[HTML]{516DDB}} \color[HTML]{F1F1F1} 0.499 & {\cellcolor[HTML]{3D50C3}} \color[HTML]{F1F1F1} 0.489 \\
B3 & {\cellcolor[HTML]{D0473D}} \color[HTML]{F1F1F1} 0.617 & {\cellcolor[HTML]{B40426}} \color[HTML]{F1F1F1} 1.000 & {\cellcolor[HTML]{86A9FC}} \color[HTML]{F1F1F1} 0.520 & {\cellcolor[HTML]{6282EA}} \color[HTML]{F1F1F1} 0.506 & {\cellcolor[HTML]{465ECF}} \color[HTML]{F1F1F1} 0.494 & {\cellcolor[HTML]{516DDB}} \color[HTML]{F1F1F1} 0.499 & {\cellcolor[HTML]{465ECF}} \color[HTML]{F1F1F1} 0.493 & {\cellcolor[HTML]{445ACC}} \color[HTML]{F1

## Protein

In [1]:
def compute_protein_stats(dataset):
    fasta_file = f"{data_dir}/{dataset}.fasta.P.sanitized"
    seqs = list(SeqIO.parse(fasta_file, "fasta"))
    seqs_len = [len(s.seq) for s in seqs]
    seqs_mean = numpy.mean(seqs_len)
    seqs_median = numpy.median(seqs_len)
    seqs_std = numpy.std(seqs_len)
    seqs_min = numpy.min(seqs_len)
    seqs_max = numpy.max(seqs_len)
    align = fasta.get_alignment(
        fasta.FastaFile.read(f"{data_dir}/trees/P/full/{dataset}/Control with Clustal Omega.fasta"))
    seqs_identity = get_sequence_identity(align)
    gaps = [seq.count("-") for seq in fasta.FastaFile.read(f"{data_dir}/trees/P/full/{dataset}/Control with Clustal Omega.fasta").values()]
    lcs_seqs = [len(lcs(s1, s2)) for (s1, s2) in combinations(seqs, 2)]
    result_df = pandas.DataFrame(
        {"median": int(seqs_median),
         "mean": seqs_mean,
         "std": round(seqs_std, 2),
         "min": seqs_min,
         "max": seqs_max,
         "identity": seqs_identity,
         "median_lcs": numpy.median(lcs_seqs),
         "mean_lcs": numpy.mean(lcs_seqs),
         "std_lcs": numpy.std(lcs_seqs),
         "min_lcs": numpy.min(lcs_seqs),
         "max_lcs": numpy.max(lcs_seqs),
         "with_gaps": sum([1 if g else 0 for g in gaps]),
         "median_gaps": numpy.median(gaps),
         "mean_gaps": numpy.mean(gaps),
         "std_gaps": numpy.std(gaps),
         "min_gaps": numpy.min(gaps),
         "max_gaps": numpy.max(gaps),
         "sample size": len(seqs_len)}, index=[dataset])
    result_df.index.name = "dataset"
    return result_df

In [7]:
dfs = []
for dataset in datasets:
    try:
        dfs.append(compute_protein_stats(dataset))
    except:
        print(dataset)
        raise
pandas.concat(dfs)

,median,mean,std,min,max,identity,median_lcs,mean_lcs,std_lcs,min_lcs,max_lcs,with_gaps,median_gaps,mean_gaps,std_gaps,min_gaps,max_gaps,sample size
dataset,,,,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,147,147.000000,0.00,147,147,0.768707,40.0,49.400000,27.570377,20,147,0,0.0,0.000000,0.000000,0,0,15
orthologs_myoglobin,154,154.000000,0.00,154,154,0.785714,33.0,59.847619,39.829824,21,154,0,0.0,0.000000,0.000000,0,0,15
orthologs_neuroglobin,151,149.600000,3.61,139,151,0.644444,113.0,109.161905,34.584093,40,151,15,4.0,5.400000,3.611094,4,16,15
orthologs_cytoglobin,205,197.866667,22.10,125,225,0.363057,67.0,79.600000,29.543705,38,190,15,31.0,38.133333,22.096355,11,111,15
orthologs_androglobin,1642,1574.866667,231.31,716,1667,0.524213,100.0,108.857143,74.755784,31,492,15,34.0,101.133333,231.308702,9,960,15
indelible,1000,1000.000000,0.00,1000,1000,0.000000,5.0,4.657692,0.653986,4,9,40,1169.0,1169.000000,0.000000,1169,1169,40
